# Y = b0 + b1X1 (72hr) (concurrent)

In [1]:
import numpy as np
import pandas as pd

# Read data

In [2]:
data=pd.read_csv('/home/luhung3080/Desktop/PycharmProjects/NCHUproject/Transformer/data_final.csv')

# Generate X Y

In [3]:
###
#cal_PMf  
###
u=np.zeros([544,5112])
for i in range (0,544):
    a=np.array(data['pm25_cal'][5112*i:5112*i+5112])
    u[i]=a.T

###
#obs_PMf
###
v=np.zeros([544,5112])
for i in range (0,544):
    a=np.array(data['pm25_obs'][5112*i:5112*i+5112])
    v[i]=a.T

print(np.shape(u))
print(np.shape(v))

(544, 5112)
(544, 5112)


In [4]:
XRestruct_Fun=u
YRestruct_Fun=v
XCov_fun=np.cov(XRestruct_Fun.T)
XMean_fun=np.zeros(5112)
YMean_fun=np.zeros(5112)
for i in range(0,5112):
    for j in range(0,544):
        XMean_fun[i]=XMean_fun[i]+(XRestruct_Fun[j][i]/544)
        YMean_fun[i]=YMean_fun[i]+(YRestruct_Fun[j][i]/544)
print(np.shape(XRestruct_Fun))
print(np.shape(YRestruct_Fun))
print(np.shape(XCov_fun))
print(np.shape(XMean_fun))
print(np.shape(YMean_fun))

(544, 5112)
(544, 5112)
(5112, 5112)
(5112,)
(5112,)


In [5]:
Xhat=XRestruct_Fun
Yhat=YRestruct_Fun
Xbar=XMean_fun
Ybar=YMean_fun
print('Xhat')
#print(Xhat)
print(np.shape(Xhat))
print('Yhat')
#print(Yhat)
print(np.shape(Yhat))
print('Xbar')
#print(Xbar)
print(np.shape(Xbar))
print('Ybar')
#print(Ybar)
print(np.shape(Ybar))

Xhat
(544, 5112)
Yhat
(544, 5112)
Xbar
(5112,)
Ybar
(5112,)


# Estimate Beta

In [6]:
G_xx=XCov_fun
G_xxt=np.zeros(5112)
for i in range(0,5112):
     G_xxt[i]=G_xx[i][i]
G_xy=0
for i in range (0,544):
    G_xyi=(Xhat[i]-Xbar)*(Yhat[i]-Ybar)
    G_xy=G_xy+G_xyi
G_xy=G_xy/543

# print(G_xxt)
print(np.shape(G_xxt))
# print(G_xy)
print(np.shape(G_xy))

(5112,)
(5112,)


In [7]:
B_1=G_xy/G_xxt
B_0=YMean_fun-(B_1*XMean_fun)

print('Beta_1')
#print(B_1)
print(np.shape(B_1))
print('Beta_0')
#print(B_0)
print(np.shape(B_0))

Beta_1
(5112,)
Beta_0
(5112,)


# Calibrating

In [8]:
after_fx=np.zeros([605,5112])
for i in range (0,605):
    a=np.array(data['pm25_cal'][5112*i:5112*i+5112])
    after_fx[i]=a.T
    
fit_fx=B_0+(B_1*after_fx)
print(np.shape(fit_fx))

(605, 5112)


# Compare

In [9]:
data0=pd.read_csv('/home/luhung3080/Desktop/PycharmProjects/Data/newdata/result.csv')
dd = data0.drop(data0[data0['TAU'] == 0].index)

In [10]:
"""
old data
"""
old_x=np.zeros([605,5112])
old_y=np.zeros([605,5112])
#data_cal
for i in range (0,605):
    a=np.array(dd['VALUE'][5112*i:5112*i+5112])
    for j in range(0,5112):
        if a[j]=='NaN' :
            a[j]=np.nan
    old_x[i]=a
#data_obs
for i in range (0,605):
    a=np.array(dd['OBS_VALUE'][5112*i:5112*i+5112])
    for j in range(0,5112):
        if a[j]=='NaN' :
            a[j]=np.nan
        if a[j]=='A' :
            a[j]=np.nan
        if a[j]=='*' :
            a[j]=np.nan
        if a[j]=='x' :
            a[j]=np.nan  
    old_y[i]=a
print(np.shape(old_x))
print(np.shape(old_y))
print(np.shape(fit_fx))

(605, 5112)
(605, 5112)
(605, 5112)


In [13]:
old_x1=np.zeros([544,5112])
old_y1=np.zeros([544,5112])
fit_fx1=np.zeros([544,5112])
for i in range (0,544):
    for j in range (0,5112):
        old_x1[i][j]=old_x[i][j]
        old_y1[i][j]=old_y[i][j]
        fit_fx1[i][j]=fit_fx[i][j]
before1=abs(old_x1-old_y1)
after1=abs(fit_fx1-old_y1)
before_error1=np.nanmean(before1)
after_error1=np.nanmean(after1)
print("544 days of training data set before error (MADE) : %f" %before_error1)
print("544 days of training data set after error (MADE) : %f" %after_error1)

544 days of training data set before error (MADE) : 7.181135
544 days of training data set after error (MADE) : 6.295365


In [14]:
old_x2=np.zeros([61,5112])
old_y2=np.zeros([61,5112])
fit_fx2=np.zeros([61,5112])
for i in range (0,61):
    for j in range (0,5112):
        old_x2[i][j]=old_x[i+544][j]
        old_y2[i][j]=old_y[i+544][j]
        fit_fx2[i][j]=fit_fx[i+544][j]
before2=abs(old_x2-old_y2)
after2=abs(fit_fx2-old_y2)
before_error2=np.nanmean(before2)
after_error2=np.nanmean(after2)
print("61 days of testing data set before error (MADE) : %f" %before_error2)
print("61 days of testing data set bafter error (MADE) : %f" %after_error2)

61 days of testing data set before error (MADE) : 6.928633
61 days of testing data set bafter error (MADE) : 6.149006
